In [159]:
# Imports
import pandas as pd
import re
import numpy as np

In [92]:
# Load data and output data stats
file_path = "./mmALL_073120_csv.csv"
# Load CSV file into a pandas DataFrame
def load_csv(file_path):
    df = pd.read_csv(file_path)
    return df

# Perform basic EDA on the DataFrame
def perform_eda(df):
    # Display the number of rows and columns in the DataFrame
    print("Shape:")
    print(df.shape)

    # Display the column names in the DataFrame
    print("Columns:")
    print(df.columns)

    # Check for missing values in the DataFrame
    print("Missing Values:")
    print(df.isnull().sum())

    # Check the data types of the columns in the DataFrame
    print("Data Types:")
    print(df.dtypes)

    # Perform additional EDA tasks as needed

# Example usage
df = load_csv(file_path)
perform_eda(df)

Shape:
(17145, 31)
Columns:
Index(['id', 'country', 'ccode', 'year', 'region', 'protest', 'protestnumber',
       'startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear',
       'protesterviolence', 'location', 'participants_category',
       'participants', 'protesteridentity', 'protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7', 'sources',
       'notes'],
      dtype='object')
Missing Values:
id                           0
country                      0
ccode                        0
year                         0
region                       0
protest                      0
protestnumber                0
startday                  1906
startmonth                1906
startyear                 1906
endday                    1906
endmonth                  1906
endyear                   1906
protester

In [93]:
# Show unique values for each column
for col in df.columns:
    print(f"For {col} we have {len(df[col].unique())} values {df[col].unique()}")

For id we have 17145 values [ 201990001  201990002  201990003 ... 9102018000 9102019000 9102020000]
For country we have 166 values ['Canada' 'Cuba' 'Haiti' 'Dominican Republic' 'Jamaica' 'Mexico'
 'Guatemala' 'Honduras' 'El Salvador' 'Nicaragua' 'Costa Rica' 'Panama'
 'Colombia' 'Venezuela' 'Guyana' 'Suriname' 'Ecuador' 'Peru' 'Brazil'
 'Bolivia' 'Paraguay' 'Chile' 'Argentina' 'Uruguay' 'United Kingdom'
 'Ireland' 'Netherlands' 'Belgium' 'Luxembourg' 'France' 'Switzerland'
 'Spain' 'Portugal' 'Germany' 'Germany West' 'Germany East' 'Poland'
 'Austria' 'Hungary' 'Czechoslovakia' 'Czech Republic' 'Slovak Republic'
 'Italy' 'Albania' 'Kosovo' 'Serbia' 'Macedonia' 'Croatia' 'Yugoslavia'
 'Bosnia' 'Serbia and Montenegro' 'Montenegro' 'Slovenia' 'Greece'
 'Cyprus' 'Bulgaria' 'Moldova' 'Romania' 'USSR' 'Russia' 'Estonia'
 'Latvia' 'Lithuania' 'Ukraine' 'Belarus' 'Armenia' 'Georgia' 'Azerbaijan'
 'Finland' 'Sweden' 'Norway' 'Denmark' 'Cape Verde' 'Guinea-Bissau'
 'Equatorial Guinea' 'Gambia' '

In [94]:
# Figure out where the missing values in some of the columns are coming from

mask = df[['startday', 'startmonth']].isna().any(axis=1)
missing_rows = df[mask]
print(missing_rows[missing_rows["country"] == "Canada"])
print(df[df["country"] == "Canada"].shape)

           id country  ccode  year         region  protest  protestnumber  \
18  201998000  Canada     20  1998  North America        0              0   
19  201999000  Canada     20  1999  North America        0              0   
24  202001000  Canada     20  2001  North America        0              0   
25  202002000  Canada     20  2002  North America        0              0   
27  202004000  Canada     20  2004  North America        0              0   
38  202013000  Canada     20  2013  North America        0              0   
39  202014000  Canada     20  2014  North America        0              0   

    startday  startmonth  startyear  ...  protesterdemand4  stateresponse1  \
18       NaN         NaN        NaN  ...               NaN             NaN   
19       NaN         NaN        NaN  ...               NaN             NaN   
24       NaN         NaN        NaN  ...               NaN             NaN   
25       NaN         NaN        NaN  ...               NaN             

In [95]:
# 
# Dealing with missing "participants" numbers.
# 

# Checking if NaN protestor_category is from empty rows

mask = df[['participants', "participants_category"]].isna().all(axis=1)
missing_rows = df[mask]
print(missing_rows["protest"].unique())

print(missing_rows[missing_rows["protest"] == 1].to_string(index=False))

# There are only 7 rows where there was a protest but the number of protesters is indeterminate.
# For now, for each case, I'll give very rough guesstimates
 
df.loc[df['id'] == 902005006, "participants"] = 500
df.loc[df['id'] == 921997001, "participants"] = 100
df.loc[df['id'] == 922001001, "participants"] = 1000
df.loc[df['id'] == 922003001, "participants"] = 1000
df.loc[df['id'] == 922006004, "participants"] = 50
df.loc[df['id'] == 931994004, "participants"] = 300
df.loc[df['id'] == 6452013004, "participants"] = 1000


[0 1]
        id     country  ccode  year          region  protest  protestnumber  startday  startmonth  startyear  endday  endmonth  endyear  protesterviolence                                               location participants_category participants            protesteridentity            protesterdemand1 protesterdemand2 protesterdemand3 protesterdemand4  stateresponse1 stateresponse2 stateresponse3 stateresponse4 stateresponse5 stateresponse6 stateresponse7                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [180]:
#
# Standardizing the number of participants based on participants and participants_category
#

# General approach: if "participants_category" exists, use that. 
# Actually, I would rather use participants and process it on my own. "participants_category" is too vague
df["participants"] = df["participants"].astype(str)
df["participants"] = df["participants"].str.lower()
df["participants"] = df["participants"].str.strip()

non_numeric_values = df["participants"][pd.to_numeric(df["participants"], errors='coerce').isna()]
# numeric_values = df["participants"][pd.to_numeric(df["participants"], errors='coerce').notna()]
print(non_numeric_values.unique())
# print(numeric_values.unique())

# Checking for all cases where a certain word appears
# print(df[df["participants"].str.contains('nearly', case=False, na=False)]["participants"].unique())

########################
# Mapping - left contains the phrase that I'm looking for. Right contains the number that I'm going to translate into
replace = {
    "hundreds of thousands": 100000,
    "hundred thousands": 100000,
    "hundreds of thousands to 1.4 million": 1000000,
    "hundreds": 100,
    "hundred": 100,
    "the civil human right front aid 110,000 people; the hong kong police aid about 20,000": 130000,
    "in northern chiang mai province, 200 kamnan and village head; in phetchaburi, about 400 kamnan and village head": 600,
    "dozen": 12,
    'march organiser and legal-sector lawmaker dennis kwok, said  2,000 people joined the protest. police put the turnout at 1,700.': 1850,
    "seven million": 7000000,
    "two million": 2000000,
    "million": 1000000,
    "tens of thousands": 10000,
    "couple thousands": 2000,
    "thousand": 1000,
}

for k,v in replace.items():
    df.loc[df["participants"].str.contains(k, case=False, na=False), 'participants'] = v


########################
# Define the substrings to remove and their replacements
replacements = {
    'more than': '', 
    '+': '', 
    "about": '',
    ">": "",
    "<": "",
    "at least": "",
    "around ": "",
    "1000s": "1000",
    "100s": "100",
    "10000s": "10000",
    "10s": "10",
    "0s": "0", ### What about just this?
    "100000s": "100000",
    "people": "",
    "protesters": "",
    "citizens": "",
    "residents": "",
    "members": "",
    "demonstrators": "",
    "some": "",
    "up to": "",
    "estimated to be": "",
    "estimated": "",
    "campaigners": "",
    "taxis": "",
    "activists": "",
    "almost": "",
    "over": "",
    "local": "",
    "villagers": "",
    "participants": "",
    "supporters": "",
    "parents": "",
    "nearly": "",
    "former estate workers":"",
    "drivers":"",
    "teachers":"",
    "construction workers":"",
    ",": "",
    " to ": "-",
    "as many as": "",
}

# Iterate over the replacements and apply the replacements to the column
for substring, replacement in replacements.items():
    df["participants"] = df["participants"].str.replace(substring, replacement, case=False)

# One 

########################
# Dealing with the ranges
# Assuming df is the DataFrame containing the data
# Define a regular expression pattern to match the range
pattern = r'(\d+)\s*-\s*(\d+)'

# Function to calculate the midpoint of a range
def calculate_midpoint(range_str):
    if type(range_str) != str:
        return range_str
    match = re.search(pattern, range_str)
    if match:
        print(range_str)
        start = int(match.group(1))
        end = int(match.group(2))
        midpoint = (start + end) // 2
        return midpoint
    else:
        return range_str

# Apply the calculation to the column
df["participants"] = df["participants"].apply(calculate_midpoint)

########################
# After this processing, things get a little dicey, so we have to handle everything manually.
replace = {
    "between 35,000 and 50,000": 42500,
    "between 100000 & 300000": 200000,
    "between 11000 and 45000": 28000,
    'the organisers said  3000 ; police said the turnout was 830 at its peak': 2000,
    "btw. 20000 & 30000": 25000,
}

for k,v in replace.items():
    df.loc[df["participants"].str.contains(k, case=False, na=False), 'participants'] = v

########################
# Of the remaining, I check if participants_category exists. If so, then use it.
replace = {'50-99': 75, '100-999': 550, '2000-4999': 3500, '1000-1999': 1500, '5000-10000': 7500, '>10000': 10000}
for k, v in replace.items():
    df.loc[(pd.to_numeric(df["participants"], errors='coerce').isna()) & (df['participants_category'] == k), "participants"] = v

########################
# Finally, we make manual guesstimates for those remaining events.
replace = {
    1302005002: 50,
    6601992002: 1000,
    6602002005: 1000,
    7321990014: 1000,
    8501998019: 1000,
    8501999009: 1000,
}

for k, v in replace.items():
    df.loc[df['id'] == k, "participants"] = v

########################
# Conversion into numbers
# Now that participants should all be string representations of numbers, we convert everything
# Starting with NaN
df["participants"] = df["participants"].replace("nan", np.nan)
df["participants"] = pd.to_numeric(df["participants"], errors='coerce')

########################
# Final checks - make sure that all valid protest data has numbers
t = df[(df["protest"] == 1)]
ts = t[pd.to_numeric(t["participants"], errors='coerce').isna()]
print(ts["participants"].unique())
# SUCCESS - empty frame.

# Below code was used to filter out those rows that had protest info but didn't have any numbers of protesters available.
# t = df[df["protest"] == 1]
# t = t[t["participants"].notna()]
# ["participants_category"]
# ts = t[pd.to_numeric(t["participants"], errors='coerce').isna()]
# print(ts[ts["participants_category"].isna()][["participants", "sources", "notes"]].to_string())

########################
# Cleanup - convert "nan" back into float(nan)


['nan']


/var/folders/8v/jp6zvvw92yx1gn1xgrd_w07r0000gn/T/ipykernel_27885/767695889.py:91: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df["participants"] = df["participants"].str.replace(substring, replacement, case=False)


[]
Empty DataFrame
Columns: [id, country, ccode, year, region, protest, protestnumber, startday, startmonth, startyear, endday, endmonth, endyear, protesterviolence, location, participants_category, participants, protesteridentity, protesterdemand1, protesterdemand2, protesterdemand3, protesterdemand4, stateresponse1, stateresponse2, stateresponse3, stateresponse4, stateresponse5, stateresponse6, stateresponse7, sources, notes]
Index: []

[0 rows x 31 columns]
